In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [54]:
arm_short_name = ['Ctr', 'AMG386', 'N', 'Ganitumab', 'MK2206', 'Ganetespib', 'VC', 'Pembro', 'TDM1/P', 'Pertuzumab']
#arm_short_name = ['Pembro']

directory_out = 'res_xgb35_analysis/'
for idx, a in enumerate(arm_short_name):
    if a == 'TDM1/P':
        b = 'TDM1P'
    else:
        b = a

    file_name = 'res_xgb35/results_xgb35_hh1norm1_' + b 
    franks = pd.read_csv(file_name + '_frank.csv')
    franks.set_index('Unnamed: 0', inplace = True)

    # remove outliers and compute ranks
    for g in franks.index.values:
        ranks = franks.loc[g].iloc[:20]
        q1 = ranks.quantile(0.25)
        q3 = ranks.quantile(0.75)
        iqr = q3 - q1 
        upper = q3 + 1.5*iqr
        upper_outliers = np.where(ranks > upper)[0]
        if upper_outliers.shape[0] > 0:
            normal_samples = [str(i) for i in list(range(20)) if i not in upper_outliers]
            r = ranks[normal_samples]
            temp=np.round([np.mean(r.values), np.std(r.values)],3)
            franks.loc[g,['rank', 'std']] = temp
        else:
            temp = np.round([np.mean(ranks.values), np.std(ranks.values)])
            franks.loc[g,['rank', 'std']] = temp
            
    franks.sort_values(by = 'rank', ascending = True, inplace = True)
    franks_q = franks.copy()    
    franks_q100 = franks_q.iloc[:100,:]
    if idx == 0:
        franks_q100_all = franks_q100.index.values.reshape(-1,1)
    else:
        franks_q100_all = np.hstack([franks_q100_all, franks_q100.index.values.reshape(-1,1)])
    
    #franks_q.to_csv(directory_out + 'franks_q_' + b + '.csv')
    franks_q100.to_csv(directory_out + 'franks_q100_' + b + '.csv')


In [58]:
df_franks_q100_all = pd.DataFrame(franks_q100_all, columns = arm_short_name)
df_franks_q100_all.to_csv(directory_out +'franks_top100genes_all.csv',index=False)

In [ ]:
#Get all protein coding genes excluding the top 100 genes
#This set will be used as background set for GO enrichement analysis
gene_table = pd.read_csv('ncbi_human_genes_protein_coding.csv')
gene_table.set_index('Symbol', inplace = True)
genes_all = gene_table.index.values
for a in arm_short_name:
    if a == 'TDM1/P':
        b = 'TDM1P'
    else:
        b = a
        
    file_name = directory_out + 'franks_q100_' + b + '.csv' 
    temp = pd.read_csv(file_name)
    g100 = temp['Unnamed: 0'].values
    g_not100 = [g for g in genes_all if g not in g100]
    print(len(g_not100))
    
    file_name = directory_out + 'franks_not100_' + b + '.csv' 
    with open(file_name, 'w') as file:
        for g in g_not100:
            file.write(g + '\n')

In [36]:
genes_all.shape

(20621,)